# Import

In [1]:
import os
import json
import pandas as pd
from collections import defaultdict
from cobra import io
from cobra.flux_analysis import flux_variability_analysis
from cobra.medium import minimal_medium

# Individual Model Simulations

In [2]:
# === CONFIGURATION ===
input_dir = "/home/arog/models/xml"            # Directory containing .xml files
output_dir = "/home/arog/models/solo_results"   # Directory to save results
fraction_of_optimum = 1.0                      # FVA at 100% of optimal growth

# === Ensure output directory exists ===
os.makedirs(output_dir, exist_ok=True)

# === Process all .xml files ===
for file_name in os.listdir(input_dir):
    if file_name.endswith(".xml"):
        model_path = os.path.join(input_dir, file_name)
        print(f"Processing: {model_path}")
        
        try:
            # === Load model ===
            model = io.read_sbml_model(model_path)
            base_name = os.path.splitext(file_name)[0]

            # === Save model summary ===
            summary_text = str(model.summary())
            summary_path = os.path.join(output_dir, f"{base_name}_summary.txt")
            with open(summary_path, "w") as f:
                f.write(summary_text)
            print(f"Saved summary: {summary_path}")

            # === Get and save minimal medium ===
            min_med = minimal_medium(model)
            minmed_path = os.path.join(output_dir, f"{base_name}_minimal_medium.csv")
            min_med.to_csv(minmed_path)
            print(f"Saved minimal medium: {minmed_path}")

            # === Run and save FVA ===
            fva_result = flux_variability_analysis(model, fraction_of_optimum=fraction_of_optimum)
            fva_path = os.path.join(output_dir, f"{base_name}_fva.csv")
            fva_result.to_csv(fva_path)
            print(f"Saved FVA: {fva_path}")



        except Exception as e:
            print(f"Failed to process {file_name}: {e}")


Processing: /home/arog/models/xml/iYL1228.xml
Saved summary: /home/arog/models/solo_results/iYL1228_summary.txt
Saved minimal medium: /home/arog/models/solo_results/iYL1228_minimal_medium.csv
Saved FVA: /home/arog/models/solo_results/iYL1228_fva.csv
Processing: /home/arog/models/xml/iSDY_1059.xml
Saved summary: /home/arog/models/solo_results/iSDY_1059_summary.txt
Saved minimal medium: /home/arog/models/solo_results/iSDY_1059_minimal_medium.csv
Saved FVA: /home/arog/models/solo_results/iSDY_1059_fva.csv
Processing: /home/arog/models/xml/iCN900.xml


Minimization of medium was infeasible.


Saved summary: /home/arog/models/solo_results/iCN900_summary.txt
Failed to process iCN900.xml: 'NoneType' object has no attribute 'to_csv'
Processing: /home/arog/models/xml/iECSE_1348.xml
Saved summary: /home/arog/models/solo_results/iECSE_1348_summary.txt
Saved minimal medium: /home/arog/models/solo_results/iECSE_1348_minimal_medium.csv
Saved FVA: /home/arog/models/solo_results/iECSE_1348_fva.csv
Processing: /home/arog/models/xml/iUMNK88_1353.xml
Saved summary: /home/arog/models/solo_results/iUMNK88_1353_summary.txt
Saved minimal medium: /home/arog/models/solo_results/iUMNK88_1353_minimal_medium.csv
Saved FVA: /home/arog/models/solo_results/iUMNK88_1353_fva.csv
Processing: /home/arog/models/xml/iETEC_1333.xml
Saved summary: /home/arog/models/solo_results/iETEC_1333_summary.txt
Saved minimal medium: /home/arog/models/solo_results/iETEC_1333_minimal_medium.csv
Saved FVA: /home/arog/models/solo_results/iETEC_1333_fva.csv
Processing: /home/arog/models/xml/iECSF_1327.xml
Saved summary: /ho

# External Reaction Analysis

In [3]:
# === CONFIGURATION ===
input_dir = "/home/arog/models/xml"               # Input directory
output_path = "/home/arog/models/solo_results/nonzero_exchange_lbs.json"  # Output JSON

# === Store results in defaultdict(list)
nonzero_lbs = defaultdict(list)

# === Process all .xml files ===
for file_name in os.listdir(input_dir):
    if file_name.endswith(".xml"):
        model_path = os.path.join(input_dir, file_name)
        model_name = os.path.splitext(file_name)[0]
        print(f"Processing: {model_path}")

        try:
            model = io.read_sbml_model(model_path)
            
            for rxn in model.exchanges:
                if rxn.lower_bound != 0:
                    nonzero_lbs[rxn.id].append((model_name, rxn.lower_bound))
        
        except Exception as e:
            print(f"Failed to process {file_name}: {e}")

# === Convert to regular dict and sort by reaction ID ===
sorted_lbs = dict(sorted(nonzero_lbs.items()))

# === Save to JSON ===
with open(output_path, "w") as f:
    json.dump(sorted_lbs, f, indent=2)

print(f"\nSaved non-zero lower bounds to: {output_path}")


Processing: /home/arog/models/xml/iYL1228.xml
Processing: /home/arog/models/xml/iSDY_1059.xml
Processing: /home/arog/models/xml/iCN900.xml
Processing: /home/arog/models/xml/iECSE_1348.xml
Processing: /home/arog/models/xml/iUMNK88_1353.xml
Processing: /home/arog/models/xml/iETEC_1333.xml
Processing: /home/arog/models/xml/iECSF_1327.xml
Processing: /home/arog/models/xml/iEcSMS35_1347.xml
Processing: /home/arog/models/xml/iEC1356_Bl21DE3.xml
Processing: /home/arog/models/xml/iECUMN_1333.xml
Processing: /home/arog/models/xml/iNRG857_1313.xml
Processing: /home/arog/models/xml/iUTI89_1310.xml
Processing: /home/arog/models/xml/ic_1306.xml

Saved non-zero lower bounds to: /home/arog/models/solo_results/nonzero_exchange_lbs.json


In [4]:
# === CONFIGURATION ===            
output_path = "/home/arog/models/solo_results/all_exchange.json"

# === Store results
r_ex_reactions = defaultdict(list)

# === Process all .xml files ===
for file_name in os.listdir(input_dir):
    if file_name.endswith(".xml"):
        model_path = os.path.join(input_dir, file_name)
        model_name = os.path.splitext(file_name)[0]
        print(f"Processing: {model_path}")

        try:
            model = io.read_sbml_model(model_path)

            for rxn in model.exchanges:  # model.exchanges gives reaction objects
                r_ex_reactions[rxn.id].append((model_name, rxn.lower_bound))

        except Exception as e:
            print(f"Failed to process {file_name}: {e}")

# === Sort and save ===
sorted_r_ex = dict(sorted(r_ex_reactions.items()))

with open(output_path, "w") as f:
    json.dump(sorted_r_ex, f, indent=2)

print(f"\nSaved R_EX reactions to: {output_path}")


Processing: /home/arog/models/xml/iYL1228.xml
Processing: /home/arog/models/xml/iSDY_1059.xml
Processing: /home/arog/models/xml/iCN900.xml
Processing: /home/arog/models/xml/iECSE_1348.xml
Processing: /home/arog/models/xml/iUMNK88_1353.xml
Processing: /home/arog/models/xml/iETEC_1333.xml
Processing: /home/arog/models/xml/iECSF_1327.xml
Processing: /home/arog/models/xml/iEcSMS35_1347.xml
Processing: /home/arog/models/xml/iEC1356_Bl21DE3.xml
Processing: /home/arog/models/xml/iECUMN_1333.xml
Processing: /home/arog/models/xml/iNRG857_1313.xml
Processing: /home/arog/models/xml/iUTI89_1310.xml
Processing: /home/arog/models/xml/ic_1306.xml

Saved R_EX reactions to: /home/arog/models/solo_results/all_exchange.json
